# Analyse von *Bluesky*-Daten, die mit `zeeschuimer` gesammelt wurden

In diesem Notebook schauen wir uns exemplarisch an, wie man automatisierte Textanalysen und Netzwerkanalysen mit *Bluesky*-Daten durchführen kann, die mit dem `R`-Paket [`atrrr`](https://jbgruber.github.io/atrrr/) gesammelt wurden.

Ausgangspunkt sind entsprechend `.csv`-Dateien mit Posts, Followers oder Followed Accounts, die mit `attr` erhoben und entsprechend in `R` exportiert wurden. Wie *Bluesky*-Daten mit `atrrr` gesammelt werden können, ist im [Beispiel-Notebook dazu](https://github.com/jobreu/atrrr-demo) erklärt.

## Datenimport

Zur Erinnerung: Die verwendete `.csv`-Datei muss über den Upload-Button (nach oben zeigender Pfeil) im File Explorer auf der linken Seite der Jupyter-Lab-Analyseumgebung hochgeladen werden.

In [ ]:
library(readr)

In [ ]:
posts <- read_csv("INSERT_FILE_NAME_HERE") # Namen der entsprechenden Datei (inkl. Dateiendung) einfügen

In [ ]:
names(posts)

In [ ]:
library(dplyr)

In [ ]:
glimpse(posts)

In [ ]:
followers <- read_csv("INSERT_FILE_NAME_HERE")

In [ ]:
names(followers)

In [ ]:
glimpse(followers)

In [ ]:
following <- read_csv("INSERT_FILE_NAME_HERE")

In [ ]:
names(following)

In [ ]:
glimpse(following)

*Hinweis*: Wenn die Daten für mehrere Accounts erhoben wurden und im kombinierten Datensatz eine (numerische) ID-Variable vergeben wurde, die anzeigt, für welchen Account die Daten gesammelt wurden (z.B. über die Funktion `map_df()` aus dem Paket `purrr`), muss diese ggf. noch in eine Textvariable mit dem entsprechenden Handle umgewandelt werden.

## Textanalyse

In [ ]:
library(quanteda)

### Erstellung eines Corpus

In [ ]:
bluesky_corpus <- posts %>% 
  distinct(uri, .keep_all = TRUE) %>% 
  select(id, uri, cid,
         author_handle, author_name,
         indexed_at, reply_count, repost_count,
         like_count, quote_count,
         is_reskeet,
         text) %>% 
  corpus(docid_field = "uri",
         text_field = "text")

### Tokenisierung & Entfernung von Stop Words

In [ ]:
tokens_bluesky <- tokens(bluesky_corpus,
                       remove_punct = TRUE,
                       remove_symbols = TRUE,
                       remove_numbers = TRUE,
                       remove_url = TRUE)

In [ ]:
tokens_bluesky <- tokens_remove(tokens_bluesky,
                              stopwords("de"))

In [ ]:
tokens_bluesky

### Document-Feature-Matrix (DFM) erstellen

In [ ]:
dfm_bluesky <- dfm(tokens_bluesky)

### Textdaten explorieren

In [ ]:
library(quanteda.textstats)

#### Worthäufigkeiten

In [ ]:
dfm_bluesky %>%
  dfm_remove(pattern = c("bsky.social", "dass",
                         "@*", "#*")) %>% # ohne User Mentions und Hashtags
  textstat_frequency(n = 20)

#### Top Hashtags

In [ ]:
dfm_tag <- dfm_select(dfm_bluesky, pattern = "#*")
toptag <- names(topfeatures(dfm_tag, 50)) # 50 häufigste Hashtags
head(toptag, 10) # 10 häufigste Hashtags

#### Top User Tags

In [ ]:
dfm_users <- dfm_select(dfm_bluesky, pattern = "@*")
topuser <- names(topfeatures(dfm_users, 50)) # 50 häufigste User Mentions
head(topuser, 10) # 10 häufigste User Mentions

## Visualisierung

In [ ]:
library(quanteda.textplots)

#### Wortwolke

In [ ]:
dfm_bluesky %>% 
  dfm_remove(pattern = c("bsky.social", "dass",
                         "@*", "#*")) %>% # ohne User Mentions und Hashtags
  dfm_trim(min_termfreq = 20) %>%
  textplot_wordcloud()

#### Plot zu Worthäufigkeiten

In [ ]:
library(ggplot2)

In [ ]:
tstat_freq <- dfm_bluesky %>% 
  dfm_remove(pattern = c("bsky.social", "dass",
                         "@*", "#*")) %>%  
  textstat_frequency(n = 20)

ggplot(tstat_freq, aes(x = frequency, y = reorder(feature, frequency))) +
  geom_col() + 
  labs(x = "Frequency", y = "Feature") +
  scale_x_continuous(expand = expansion(mult = c(0, 0.05)))

### Relative Häufigkeiten / Keyness

Mit einer sog. [Keyness-Analyse](https://tutorials.quanteda.io/statistical-analysis/keyness/) kann man die Häufigkeit von Wörtern zwischen Ziel- und Referenzdokumenten vergleichen. Für unseren Anwendungsbereich könnte das z.B. eine bestimmte Partei oder ein:e bestimmte:r Politiker:in im Vergleich zu anderen Parteien oder Politiker:innen sein.

In [ ]:
tstat_key <- dfm_bluesky %>% 
  dfm_remove(pattern = c("bsky.social", "dass",
                         "@*", "#*")) %>%  
  textstat_keyness(target = dfm_bluesky$author_handle == "insert_user_handle") # hier den Namen des zu vergleichenden Accounts einfügen

textplot_keyness(tstat_key)

## Netzwerkanalyse

Je nachdem, welche *Bluesky*-Daten wir gesammelt haben bzw. nutzen, lassen sich unterschiedliche Netzwerke erstellen: Repost-, Follower- oder Following-Netzwerke. In allen Fällen sind die Verbindungen (Edges) zwischen den Accounts (Nodes) gerichtet.

In [ ]:
library(igraph)

### Repost-Netzwerk

Im Falle des Reposts-Netzwerks sind die Verbindungen (Edges) auch gewichtet. Das Gewicht (Weight) stellt hier die Anzahl an Reposts dar.

In [ ]:
reposts <- posts %>%
  filter(is_reskeet == TRUE) %>%
  select(source = id, target = author_handle) %>% 
  count(source, target, name = "weight") %>% 
  filter(source != target) %>%
  filter(weight >= 2)

In [ ]:
head(reposts)

In [ ]:
g1 <- repost_network <- graph_from_data_frame(reposts,
                                          directed = TRUE)

In [ ]:
plot(g1, 
     edge.width = E(g1)$weight * 2, # dickere Edges für größere Gewichte (mehr Reposts)
     edge.arrow.size = 0.5,
     edge.label = E(g1)$weight,
     main = "Weighted Repost Network")

### Follower-Netzwerk

In [ ]:
follower_net <- followers %>%
  select(source = actor_handle, target = id)

In [ ]:
g2 <- follower_network <- graph_from_data_frame(follower_net,
                                              directed = TRUE)

In [ ]:
g2

Wenn Daten zu vielen Followern für mehrere (insb. "reichweitenstarke") Accounts gesammelt werden, wird ein Plot mit `igraph` schnell unübersichtlich. Hier bieten sich ggf. Alternativen zur interaktiven Visualisierung wie das `R`-Paket [`visNetwork`](https://datastorm-open.github.io/visNetwork/) an.

### Following-Netzwerk

In [ ]:
following_net <- following %>%
  select(source = id, target = actor_handle)

In [ ]:
g3 <- following_network <- graph_from_data_frame(following_net,
                                                 directed = TRUE)

In [ ]:
g3

Auch beim Following-Netzwerk werden statische Plots mit `igraph` schnell unübersichtlich.